In [59]:
import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split

## Import Data

In [60]:
with open("student-handbook2023-24.pdf", 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    text_corpus = []
    for page in reader.pages:
        text_corpus.append(page.extract_text())

full_text = ' '.join(text_corpus)  # Combine text from all pages into a single string

In [61]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [62]:
data = {"text": full_text}
dataset = Dataset.from_dict(data)

# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/106258 [00:00<?, ? examples/s]

In [65]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
)

In [66]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [67]:
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: start_logits,end_logits. For reference, the inputs it received are input_ids,attention_mask.

In [52]:
# Access the train and validation datasets
train_dataset = dataset_split['train']
val_dataset = dataset_split['test']

In [42]:
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",       # Save after each epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True
)

In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [55]:
trainer.train()

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [44]:
# After training, save the model
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('my_custom_qa_model\\tokenizer_config.json',
 'my_custom_qa_model\\special_tokens_map.json',
 'my_custom_qa_model\\vocab.txt',
 'my_custom_qa_model\\added_tokens.json')

In [ ]:
# Test the model with a question
question = "What is the university's grading policy?"

inputs = tokenizer(question, return_tensors="pt")
outputs = model(**inputs)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Get the answer from the model
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

print(f"Answer: {answer}")

## Test the Model

In [ ]:
qa_pipeline = pipeline("question-answering", model="my_custom_qa_model", tokenizer="my_custom_qa_model")

In [ ]:
# List of questions
questions = [
    "What are the requirements for maintaining academic integrity?",
    "What is the attendance policy for IBA students?",
    "What are the prerequisites for students applying for the Data Science program?",
    "How are student assessments and grading conducted?",
    "What are the consequences for violating student policies?"
]

In [ ]:
# Ask each question without specifying additional context
for question in questions:
    result = qa_pipeline(question)  # No need to pass context if model is fine-tuned on it
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}\n")